# 第9章　摄像头模型和增强现实

In [ ]:
import math
import timeit
import cv2
import numpy as np
gray_img = cv2.cvtColor(bg_img, cv2.COLOR_BGR2GRAY)

def convert_to_gray(src, dst=None):
    weight = 1.0 / 3.0
    m = np.array([weight, weight, weight], np.float32)
    return cv2.transform(src, m, dst)

def map_point_onto_plane(point_2D, image_size, image_scale):
    x, y = point_2D
    w, h = image_size
    return (image_scale * (x-0.5*w), image_scale * (y-0.5*h), 0.0)

def map_points_to_plane(points_2D, image_size, image_real_height):
    w, h = image_size
    image_scale = image_real_height / h
    points_3D = [map_point_onto_plane(point_2D, image_size, image_scale) for point_2D in points_2D]
    return np.array(points_3D, np.float32)

def map_vertices_to_plane(image_size, image_real_height):
    w, h = image_size
    vertices_2D = [(0, 0), (w, 0), (w, h), (0, h)]
    vertex_indices_by_face = [[0, 1, 2, 3]]
    vertices_3D = map_points_to_plane(vertices_2D, image_size, image_real_height)
    return vertices_3D, vertex_indices_by_face

class ImageTrackingDemo():

    def __init__(self, capture, diagonal_fov_degrees=70.0, target_fps=25.0,
                reference_image_path="images/reference_image.png",
                reference_image_real_height=1.0):
        self._capture = capture
        success, trail_image = capture.read()
        if success:
            h , w = trail_image.shape[:2]
        else:
            w = capture.get(cv2.CAP_PROP_FRAME_WIDTH)
            h = capture.get(cv2.CAP_PROP_FRAME_HEIGHT)
        self._image_size = (w, h)
        diagonal_image_size = (w ** 2.0 + h ** 2.0) ** 0.5
        diagonal_fov_radians = diagonal_fov_degrees * math.pi / 180.0
        focal_length = 0.5 * diagonal_image_size / math.tan(0.5 * diagonal_fov_radians)
        self._camera_matrix = np.array([[focal_length, 0.0, 0.5 * w], [0.0, focal_length, 0.5 * h], [0.0, 0.0, 1.0]], np.float32)
        self._distortion_coefficients = None
        self._kalman = cv2.KalmanFilter(18, 6)
        self._kalman.processNoiseCov = np.identity(18, np.float32) * 1e-5
        self._kalman.measurementNoiseCov = np.identity(6, np.float32) * 1e-2
        self._kalman.errorCovPost = np.identity(18, np.float32)
        self._kalman.measuremnetMartrix = np.array(
            [
                [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
                 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
                 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
                 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
                 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
                 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
                 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            ], np.float32
        )
        self._init_kalman_transition_martrix(target_fps)
        self._was_tracking = False
        self._reference_image_real_height = reference_image_real_height
        reference_axis_length = 0.5 * reference_image_real_height
        self._bgr_image = None
        self._gray_image = None
        self._mask = None
        patchSize = 31
        self._feature_detector = cv2.ORB_create(nfeatures=250, scaleFactor=1.2, nlevels=16, edgeThreshold=patchSize, patchSize=patchSize)
        bgr_reference_image = cv2.imread(reference_image_path, cv2.IMREAD_COLOR)
        reference_image_h, reference_image_w = bgr_reference_image.shape[:2]
        reference_image_resize_factor = (2.0 * h) / reference_image_h
        bgr_reference_image = cv2.resize(bgr_reference_image, (0, 0), None, reference_image_resize_factor, reference_image_resize_factor, cv2.INTER_CUBIC)
        gray_reference_image = convert_to_gray(bgr_reference_image)
        reference_mask = np.empty_like(gray_reference_image)
        reference_keypoints = []
        self._reference_descriptors = np.empty((0, 32), np.uint8)
        num_segments_y = 6
        num_segments_x = 6
        for segment_y, segment_x in np.ndindex((num_segments_y, num_segments_x)):
            y0 = reference_image_h * segment_y // num_segments_y - patchSize
            x0 = reference_image_w * segment_x // num_segments_x - patchSize
            y1 = reference_image_h * (segment_y + 1) //num_segments_y + patchSize
            x1 = reference_image_w * (segment_x + 1) //num_segments_x + patchSize
            reference_mask.fill(0)
            cv2.rectangle(reference_mask, (x0, y0), (x1, y1), 255, cv2.FILLED)
            more_reference_keypoints, more_reference_descriptors = self._feature_detector.detectAndCompute(gray_reference_image, reference_mask)
            if more_reference_descriptors is None:
                continue
            reference_keypoints += more_reference_keypoints
            self._reference_descriptors = np.vstack((self._reference_descriptors, more_reference_descriptors))
            cv2.drawKeypoints(gray_reference_image, reference_keypoints, bgr_reference_image, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        ext_i = reference_image_path.rfind('.')
        reference_image_keypoints_path = reference_image_path[:ext_i] + "_keypoints" + reference_image_path[ext_i:]
        cv2.imwrite(reference_image_keypoints_path, bgr_reference_image)
        FlANN_INDEX_LSH = 6
        index_params = dict(algorithm=FlANN_INDEX_LSH, table_number=6, key_size=12, multi_probe_level=1)
        search_params = dict()
        self._descriptor_matcher = cv2.FlannBasedMatcher(index_params, search_params)       